# Demand Forecasting (AI, Data Innovation Team, CJ Express)

## File: Training Model (Cluster Model)

 C.J. Express Group Co.,Ltd. All Rights Reserved.
 
 Author: Kao Panboonyuen, Ph.D.
 
 Year: 2021
 ![](https://github.com/kaocj/demandforecasting_darkside/raw/main/demand_main.PNG)

In [1]:
!rm -rf cjdarkside.py
!wget https://github.com/kaocj/demandforecasting_darkside/raw/main/cjdarkside.py -q --show-progress

cjdarkside.py       100%[===================>]  38.16K  --.-KB/s    in 0.001s  


In [2]:
spark

# Import Library

In [3]:
from cjdarkside import *

# Name your CJ DARKSIDE model

In [4]:
version = '3_FULL_NEARMISS'

gs_model = "gs://cj-demand-forecasting-dataset/model/2021-March-v1/"
gs_csv = "gs://cj-demand-forecasting-dataset/csv/"

PL_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_c_binary_model_2021_v"+str(version)+".plmodel"
ML_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_ml_c_binary_model_2021_v"+str(version)+".model"

PL_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_r_binary_model_2021_v"+str(version)+".plmodel"
ML_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_ml_r_binary_model_2021_v"+str(version)+".model"

# CREATE TEST SET

In [8]:
df = spark.read.format("csv").option("header", "true").load(YOUR_DATASET_PATH)

print('-----------------------------------')
print('-- welcome to the darkside of cj --')
print('-----------------------------------')

df.cache()

-----------------------------------
-- welcome to the darkside of cj --
-----------------------------------
count: before sampling:  1372654452
count: after sampling:  1372654452


'\nsampling = 0.3\ncount: before sampling:  1372,654,452\ncount: after sampling:  411,761,580\n\nsampling = 0.03\ncount: before sampling:  1,372,654,452\ncount: after sampling:  41,761,580\n\nsampling = 0.003\ncount: before sampling:  1,372,654,452\ncount: after sampling:  4,761,580\n\n'

In [9]:
df = df.select("*").withColumn("cj_darkside_id", monotonically_increasing_id())

df = df.withColumnRenamed("Date","SalDate")
df = df.withColumn('DayofMonth', F.dayofmonth(F.col('SalDate')))

df = extract_date(df)

df = df.dropna(how='all')

In [14]:
df = df.withColumnRenamed("types","promotion")
df = one_hot_encoder_promotion(df,'promotion')

+----------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+--------------+----------+-------+-----+---------+----+-------+----------+------------+-------------+---------------+
|   SalDate|BranchCode|MaterialCode|avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|cj_darkside_id|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|promotion_idx|  promotion_vec|
+----------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+--------------+----------+-------+-----+---------+----+-------+----------+------------+-------------+---------------+
|2020-01-01|      0002|    20006726|       15.0|      30|      30|    1|          30|           1|    BG003|             2|         1|      1|    1|        4|2020|      1|         1|           0|          1.0|(839,[1],[1.0])|
|2020-01-01|      0002|    20007292|       10.0|      15|      15|    1|          45|           

# Main (Feature Engineering)

In [15]:
# SAME AS PREVIOUS MODEL

# PUT YOUR CODE OR STEP HERE

In [16]:
final_df = clean_binary(df)
final_df.cache()

print('now features have: ', final_df.columns)
print('now show of data: ', final_df.show(2))
print('now count of data: ', final_df.count())

now features have:  ['SalDate', 'BranchCode', 'MaterialCode', 'avgPriceDis', 'avgPrice', 'supPrice', 'label', 'totalNetSale', 'TotalQtySale', 'promotion', 'cj_darkside_id', 'DayofMonth', 'Yearday', 'Month', 'DayofWeek', 'Year', 'Quarter', 'WeekOfYear', 'MonthQuarter', 'promotion_idx', 'promotion_vec']
+----------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+--------------+----------+-------+-----+---------+----+-------+----------+------------+-------------+---------------+
|   SalDate|BranchCode|MaterialCode|avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|cj_darkside_id|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|promotion_idx|  promotion_vec|
+----------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+--------------+----------+-------+-----+---------+----+-------+----------+------------+-------------+---------------+
|2020-01-01|       

In [18]:
count_column_types(final_df)

type  count  \
0  bigint  1       
1  double  5       
2  int     12      
3  string  2       
4  vector  1       

                                                                                                                                      names  
0  cj_darkside_id                                                                                                                            
1  supPrice | promotion_idx | totalNetSale | avgPriceDis | avgPrice                                                                          
2  Year | TotalQtySale | Month | BranchCode | Quarter | WeekOfYear | Yearday | DayofWeek | label | MonthQuarter | DayofMonth | MaterialCode  
3  promotion | SalDate                                                                                                                       
4  promotion_vec

In [19]:
# check missing value
final_df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in final_df.columns)).show()

+-------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+--------------+----------+-------+-----+---------+----+-------+----------+------------+-------------+-------------+
|SalDate|BranchCode|MaterialCode|avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|cj_darkside_id|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|promotion_idx|promotion_vec|
+-------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+--------------+----------+-------+-----+---------+----+-------+----------+------------+-------------+-------------+
|      0|         0|           0|          0|       0|       0|    0|           0|           0|        0|             0|         0|      0|    0|        0|   0|      0|         0|           0|            0|            0|
+-------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+-----------

# GBT ML Model (Classifier)

In [21]:
############ ----- LOAD ML MODEL ----- ############

read_plmodel = PipelineModel.read().load(PL_MODEL_Classification_PATH)
print (read_plmodel.stages)

read_model = GBTRegressor.read().load(ML_MODEL_Classification_PATH)

############ ----- DONE ML MODEL ----- ############

predictionsC = read_plmodel.transform(final_df)
predictionsC.cache()

print("TESTING Process: Done")

[VectorAssembler_90f6f50b9f80, GBTClassificationModel (uid=GBTClassifier_d12fca19d844) with 15 trees]
Training Process: Done


# GBT ML Model (Regressor)

In [ ]:
############ ----- LOAD ML MODEL ----- ############

read_plmodel = PipelineModel.read().load(final_df)
print (read_plmodel.stages)

read_model = GBTRegressor.read().load(ML_MODEL_Regression_PATH)

############ ----- DONE ML MODEL ----- ############

predictionsR = read_plmodel.transform(final_df)
predictionsR.cache()

print("Testing Process: Done")

Number of Training Set:  223315813
Number of Testing Set:  95693035
VectorAssembler_956e14378119
GBTRegressor_90fa834d77b6


In [ ]:
predictionsC.show(2)

In [ ]:
predictionsR.show(2)

In [ ]:
def ansByProb(p, t=0.4):
  if p>=t: return 1
  elif p<t: return 0
  else: 0

predictionsA = predictionsC.join(predictionsR,(predictionsC.SalDate == predictionsR.SalDate)&\
                  (predictionsC.BranchCode == predictionsR.BranchCode)&\
                  (predictionsC.MaterialCode == predictionsR.MaterialCode)).select(predictionsC["*"],predictionsR["predict_qty_lv2"])

split1_udf = udf(lambda value: value[0].item(), FloatType())
split2_udf = udf(lambda value: value[1].item(), FloatType())

predictionsA = predictionsA.withColumn('probability_class0', split1_udf('probability')).withColumn('probability_class1', split2_udf('probability'))

ansByP = F.udf(ansByProb, IntegerType())
predictionsA = predictionsA.withColumn("predict_bi_lv1_with_thresh", ansByP("probability_class1"))

predictionsA = predictionsA.withColumn("predict_qty_final", col("predict_bi_lv1")*col("predict_qty_lv2"))
#predictionsA = predictionsA.withColumn("predict_qty_final", col("predict_bi_lv1_with_thresh")*col("predict_qty_lv2"))

In [ ]:
predictionsA.show(2)

# POST PROCESSING WITH TIME SERIES (CR.TON)

In [ ]:
# predictionsA = post_score_v3(predictionsA)
# predictionsA.show(2)